# Airline Customer Feedback Analysis

### Business Understanding

The airline industry is highly competitive and customer service is a critical area that can influence customer loyalty and brand reputation. Timely and effective handling of customer feedback is crucial for maintaining service quality. Analyzing customer feedback can provide insights into common concerns and areas for improvement.

### Project Goals

The primary goal of this project is twofold:

1. **Sentiment Analysis**: To automatically determine the sentiment of customer comments. The sentiment can be:
   - **Negative (Complaint)**: Indicates the customer faced an issue or is dissatisfied.
   - **Positive (Congratulation)**: Reflects customer satisfaction or a positive experience.
   - **Neutral (Opinion)**: Represents a customer's opinion that is neither a direct complaint nor praise.

2. **Complaint Classification**: To classify customer feedback into categories for routing to the appropriate department. This classification will help in streamlining the process of addressing customer feedback efficiently. Example categories include:
   - **Luggage Issues**: For complaints or feedback related to damaged or lost luggage.
   - **Flight Delays**: For issues pertaining to delayed or cancelled flights.
   - **Billing Issues**: For concerns related to overcharges, refunds, or payment discrepancies.

The end product will be a model or a set of models that can accurately tag incoming customer comments with the correct sentiment and category, allowing the customer service team to promptly address and resolve the issues.

In [7]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import nltk
from nltk.corpus import stopwords
import os.path

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import little_mallet_wrapper as lmw


# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline


# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

path_to_mallet = 'mallet-2.0.8/bin/mallet'  # CHANGE THIS TO YOUR MALLET PATH

In [8]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/gabot/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/gabot/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
# Load data
data = pd.read_excel('output.xlsx', engine='openpyxl')

# Change typo in column name
data.rename(columns={'descripciom': 'descripcion'}, inplace=True)


In [17]:
# Convert to list
data = data.descripcion.values.tolist()

# Remove non-string elements
data = [str(sent) for sent in data]

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['Gracias por escribirnos, hemos recibido tu correo y en seguida daremos '
 'respuesta. Para emergencias, por favor comunicate al 4777920450 ']


In [18]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['gracias', 'por', 'escribirnos', 'hemos', 'recibido', 'tu', 'correo', 'en', 'seguida', 'daremos', 'respuesta', 'para', 'emergencias', 'por', 'favor', 'comunicate', 'al']]


In [19]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['gracias', 'por', 'escribirnos_hemos_recibido', 'tu', 'correo', 'en', 'seguida_daremos_respuesta', 'para', 'emergencias', 'por', 'favor', 'comunicate', 'al']


In [21]:
#Define stop words
stop_words_en = set(stopwords.words('english'))
stop_words_sp = set(stopwords.words('spanish'))

# Add additional Spanish stopwords
new_stopwords = ["buenas", "buen","blueaerolinea","saludos cordiales","com mx","www","gmail com","mailto","gmail", "com","hotmail","image004","hola","tarde","gmail com","muchas","gracias", "buenos","día","cid","mx","ustede","correo", "electronico",
                          "saludos","cordiales", "formato","pdf","solo","ustedes","quedo","espera","noche", "blue", "aerobu","adjunto podrá", "términos", "condiciones","correo electrónico","saber", "si","podrás encontrar","quedo","espera","noche",
                          "adjunto","podrá", "png","podrás","encontrar","grupos","quedo","favor","hoy", "quisiera", "tardes","adjunto podrá", "términos condiciones","correo electrónico","saber", "si","image","6c756af0","noches","01d57aa3",
                      "com", "nbsp","cognitoforms","dia","quedo","gracias","vea","saludos","favor", "días","hola","image004","xx ","tarde","gmail com","muchas","gracias", "buenos","día","cid","mx","ustede","puedo", "hacer"]

# Combine all stopwords
all_stopwords = stop_words_en.union(stop_words_sp).union(new_stopwords)

# Define stopwords excluding certain words if needed
not_stopwords = {} 
final_stop_words = set([word for word in all_stopwords if word not in not_stopwords])
print(final_stop_words)

{'hasta', 'fueses', 'vuestras', 'estuvieses', 'again', 'we', 'and', 'this', 'y', 'tenéis', 'habías', 'our', 'off', 'sentidos', 'seáis', 'nosotras', 'serán', 'todo', 'es', 'estuvierais', 'nor', 'adjunto', 'estabais', 'sí', 'you', 'sin', 'contra', 'estés', 'a', 'hubiera', 'estén', 'tenidas', 'habidas', 'mustn', 'eres', 'does', 'esos', 'hola', 'won', 'suyo', 'estuviese', 'hayan', 'entre', 'seré', 'un', 'blueaerolinea', 'vuestra', 'tuviese', 'te', 'términos condiciones', 'tenido', 'él', 'estarías', 'habida', "won't", 'at', 'to', 'les', 'himself', 'durante', 'tus', 'fueron', 't', 'habría', '6c756af0', 'estuve', 'estuviera', 'through', 'teníamos', 'tendréis', 'seamos', 'sentidas', 'who', 'having', 'hadn', 'habíais', 'tenemos', 'tuve', "you've", 'png', 'hubieras', 'same', 'if', 'mucho', "weren't", 'fuese', 'tendría', 'tuviera', 'here', 'era', 'tienes', 'nosotros', 'estuvieras', 'nuestros', 'estamos', 'hubiésemos', 'while', 'do', 'more', 'ha', 'can', "shan't", 'what', 'themselves', 'my', 'now'

In [22]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    # return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    return [[word for word in simple_preprocess(str(doc)) if word not in final_stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [23]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [24]:
# Create Dictionary
id2word = corpora.Dictionary(data_words_bigrams)

# Create Corpus
texts = data_words_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [25]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('comunicate', 1),
  ('emergencias', 1),
  ('escribirnos', 1),
  ('recibido', 1),
  ('respuesta', 1),
  ('seguida_daremos', 1)]]

In [26]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Save LDA model
lda_model.save("lda_model")

# Optionally, you can also save the corpus and id2word dictionary for later use
# Save corpus
gensim.corpora.MmCorpus.serialize('corpus.mm', corpus)

# Save id2word dictionary
id2word.save('id2word.dict')